In [56]:
from pyspark.sql import Row, DataFrame
import json

from xinghe.spark import *
from app.common.json_util import *
from xinghe.s3 import *
from app.common.json_util import json_loads
from app.common.s3.client import get_s3_client, list_s3_objects
from app.common.s3.read import read_s3_rows
from app.common.s3.write import S3DocWriter

from lxml.html import HtmlElement, HTMLParser, fromstring, tostring, document_fromstring
from lxml import etree
from lxml.html import fromstring as str2html
import os
try:
    from llm_web_kit.pipeline.extractor.html.magic_html import GeneralExtractor
    from llm_web_kit.libs.html_utils import html_to_element, element_to_html
except:
    from magic_html import GeneralExtractor

In [5]:
s3_path = "s3://puyu3-users/wangrui/puyu3-cc-recall-html/cc-html-0218/batch-filter-5-subject-domain-0219/v001/batch-0/0-physics/part-67b4df819443-000010.jsonl"

In [67]:
i = 0
total = 50 # 测试50个
save_dir = "./html-md"
os.makedirs(save_dir, exist_ok=True)
start_i = 0


def __clear_html(html_source:str) -> str:
    """
    清理html里的无关tag
    """
    new_html_source = html_source.replace(' ', '').replace('\n', '').replace('\t', '')
    try:
        # 解析HTML
        root = etree.HTML(new_html_source)
        # 删除所有的<script>和<style>元素
        for element in root.xpath('//*[self::script or self::style]'):
            element.getparent().remove(element)
        # 打印清除后的HTML
        return etree.tostring(root, pretty_print=True).decode()
    except Exception as e:
        return new_html_source
        logger.exception(e)



for row in read_s3_rows(s3_path):
    if i>=total+start_i:
        break
    i += 1
    if i<start_i:
        continue
    
    data = json_loads(row.value)  # dict_keys(['id', 'track_id', 'domain', 'url', 'date', 'content_length', 'status', 'site_type', 'content_charset', 'tag', 'html', 'raw_warc_path', 'cc_dump', 'sub_path', 'filelist_name'])
    html_source = data.get("html", "")
    url = data.get("url", '')
    
    with open(f"{save_dir}/{i}_raw.html", 'w', encoding='utf-8') as f:
        f.write(html_source)

    # 将HTML字符串转换为UTF-8字节序列
    try:
        root = html_to_element(html_source)
    except Exception as e:
        print(e)
        continue
    
    # # 移除所有的<script>和<style>元素，任何不想要的都可以加入
    for element in root.xpath("//script|//style"):
        element.getparent().remove(element)
    cleaned_html = element_to_html(root) # 得到清理之后的标签

    with open(f"{save_dir}/{i}_cleaned.html", 'w', encoding='utf-8') as f:
        f.write(cleaned_html)
    
    # 接下来提取正文，无论用任何方法，返回一个正文的html主体。这里使用magic-html
    extractor = GeneralExtractor()
    rtl = extractor.extract(cleaned_html, base_url=url, )
    main_html = rtl['html']


    with open(f"{save_dir}/{i}_main.html", "w", encoding='utf-8') as f:
        f.write(main_html)

    # 最后把main_html转成md,然后保存到文件里，看一下是否正常
    
    
    from app.format.html.html2text import html_to_text
    md = html_to_text(main_html, as_md=True, )
    
    with open(f"./{save_dir}/{i}.md", 'w', encoding='utf-8') as f:
        f.write(md)


Unicode strings with encoding declaration are not supported. Please use bytes input or XML fragments without declaration.


In [23]:

from llm_web_kit.input.datajson import DataJson, StructureMapper
def clist_filter_factory(self, field_you_want = ('paragraph')):
    """把content_list转化为md格式，只接受 Node 类型为 ['a', 'b'].
    Returns:
        str: md格式的文本内容
    """
    md_blocks = []  # 每个是个DocElementType规定的元素块之一转换成的文本
    content_lst = self._get_data()
    for page in content_lst:
        for content_lst_node in page:
            if content_lst_node['type'] in field_you_want:
                txt_content = self._StructureMapper__content_lst_node_2_md(content_lst_node)
                if txt_content and len(txt_content) > 0:
                    md_blocks.append(txt_content)
    md = self._StructureMapper__md_para_splitter.join(md_blocks)
    md = md.strip() + self._StructureMapper__text_end  # 加上结尾换行符
    return md
def clist_to_c(d_iter):
    StructureMapper.clist_filter = clist_filter_factory
    for d in d_iter:
        try:
            prompt = ''
            data_id = d['track_id']
            content  = DataJson(d).get_content_list().clist_filter()
            new_d = {
                'track_id': data_id,
                'prompt': prompt,
                'content_qa': content,
            }
            yield new_d
        except Exception as e:
            raise e

In [24]:
data = {
  "track_id": "fa9dde56-154c-4c43-8584-c735469ede41",
  "url": "https://new.qq.com/omn/20230629/20230629A0A3ZU00",
  "html": "<body> \n <p>【环球时报-环球网报道 记者 邢晓婧】28日，外交部发言人毛宁主持例行记者会。有记者提问称，近日，韩国媒体报道称，匿名知情人士称日本政府提前获得了国际原子能机构福岛核污染水处置技术工作组最终评估报告草案，并提出实质性修改意见，对最终报告结论施加了不当影响。日本官员给了机构秘书处工作人员100余万欧元。请问中方对此有何评论？</p> \n <p>毛宁对此表示，中方高度关注有关报道。日本政府有责任对相关报道给出令人信服的解释，国际原子能机构秘书处也应该作出回应。事实上，报道加剧了国际社会对日本核污染水排海的担忧，人们有理由质疑国际原子能机构技术工作组最终评估报告的结论是否公正客观。</p> \n <p>毛宁称，中方在这个问题上的立场是一贯的，核污染水排海关乎国际社会的共同利益，不是日方一家的私事。中方敦促日方正视国际社会和国内民众的关切，停止强行推动排海计划，切实以科学、安全、透明的方式处置核污染水，并接受严格的国际监督。</p> \n <p>毛宁表示，中方希望国际原子能机构秘书处秉持客观、专业、公正的原则，充分尊重和采纳技术工作组各方专家的意见，拿出一份经得起科学和历史检验的评估报告，不要为日方的排海方案背书，国际社会将拭目以待。</p> \n</body>",
  "page_layout_type": "article",
  "domain": "new.qq.com",
  "dataset_name": "tencent",
  "data_source_category": "HTML",
  "meta_info": {
    "filename": "s3://private-crawl-data/zh-web-tencent/20241218_p1/2023/1732804785928.json.gz"
  },
  "content_list": [
    [
      {
        "type": "paragraph",
        "raw_content": "<div><div><p>【环球时报-环球网报道 记者 邢晓婧】28日，外交部发言人毛宁主持例行记者会。有记者提问称，近日，韩国媒体报道称，匿名知情人士称日本政府提前获得了国际原子能机构福岛核污染水处置技术工作组最终评估报告草案，并提出实质性修改意见，对最终报告结论施加了不当影响。日本官员给了机构秘书处工作人员100余万欧元。请问中方对此有何评论？</p></div></div>",
        "content": [
          {
            "c": "【环球时报-环球网报道 记者 邢晓婧】28日，外交部发言人毛宁主持例行记者会。有记者提问称，近日，韩国媒体报道称，匿名知情人士称日本政府提前获得了国际原子能机构福岛核污染水处置技术工作组最终评估报告草案，并提出实质性修改意见，对最终报告结论施加了不当影响。日本官员给了机构秘书处工作人员100余万欧元。请问中方对此有何评论？",
            "t": "text"
          }
        ]
      },
      {
        "type": "url",
        "raw_content": "<div><div><p>毛宁对此表示，中方高度关注有关报道。日本政府有责任对相关报道给出令人信服的解释，国际原子能机构秘书处也应该作出回应。事实上，报道加剧了国际社会对日本核污染水排海的担忧，人们有理由质疑国际原子能机构技术工作组最终评估报告的结论是否公正客观。</p></div></div>",
        "content": [
          {
            "c": "毛宁对此表示，中方高度关注有关报道。日本政府有责任对相关报道给出令人信服的解释，国际原子能机构秘书处也应该作出回应。事实上，报道加剧了国际社会对日本核污染水排海的担忧，人们有理由质疑国际原子能机构技术工作组最终评估报告的结论是否公正客观。",
            "t": "text"
          }
        ]
      },
      {
        "type": "paragraph",
        "raw_content": "<div><div><p>毛宁称，中方在这个问题上的立场是一贯的，核污染水排海关乎国际社会的共同利益，不是日方一家的私事。中方敦促日方正视国际社会和国内民众的关切，停止强行推动排海计划，切实以科学、安全、透明的方式处置核污染水，并接受严格的国际监督。</p></div></div>",
        "content": [
          {
            "c": "毛宁称，中方在这个问题上的立场是一贯的，核污染水排海关乎国际社会的共同利益，不是日方一家的私事。中方敦促日方正视国际社会和国内民众的关切，停止强行推动排海计划，切实以科学、安全、透明的方式处置核污染水，并接受严格的国际监督。",
            "t": "text"
          }
        ]
      },
      {
        "type": "paragraph",
        "raw_content": "<div><div><p>毛宁表示，中方希望国际原子能机构秘书处秉持客观、专业、公正的原则，充分尊重和采纳技术工作组各方专家的意见，拿出一份经得起科学和历史检验的评估报告，不要为日方的排海方案背书，国际社会将拭目以待。</p></div></div>",
        "content": [
          {
            "c": "毛宁表示，中方希望国际原子能机构秘书处秉持客观、专业、公正的原则，充分尊重和采纳技术工作组各方专家的意见，拿出一份经得起科学和历史检验的评估报告，不要为日方的排海方案背书，国际社会将拭目以待。",
            "t": "text"
          }
        ]
      }
    ]
  ]
}
a= clist_to_c([data])
print(a)

<generator object clist_to_c at 0x733a9020c2e0>


In [25]:
for i in a :
    print(i)

{'track_id': 'fa9dde56-154c-4c43-8584-c735469ede41', 'prompt': '', 'content_qa': '【环球时报-环球网报道 记者 邢晓婧】28日，外交部发言人毛宁主持例行记者会。有记者提问称，近日，韩国媒体报道称，匿名知情人士称日本政府提前获得了国际原子能机构福岛核污染水处置技术工作组最终评估报告草案，并提出实质性修改意见，对最终报告结论施加了不当影响。日本官员给了机构秘书处工作人员100余万欧元。请问中方对此有何评论？\n\n毛宁称，中方在这个问题上的立场是一贯的，核污染水排海关乎国际社会的共同利益，不是日方一家的私事。中方敦促日方正视国际社会和国内民众的关切，停止强行推动排海计划，切实以科学、安全、透明的方式处置核污染水，并接受严格的国际监督。\n\n毛宁表示，中方希望国际原子能机构秘书处秉持客观、专业、公正的原则，充分尊重和采纳技术工作组各方专家的意见，拿出一份经得起科学和历史检验的评估报告，不要为日方的排海方案背书，国际社会将拭目以待。\n'}
